In [1]:
import pandas as pd
import altair as alt

import assets

## WOW 2018
### Week 01: Looks vs. Personality

In [2]:
df08w01 = pd.read_csv(assets.DATA_18W01)
df08w01.head()

,Nationality,Gender,Value,Measure
0,Danish,Men,506,Unweighted Sample
1,Danish,Men,504,Weighted Sample
2,Danish,Men,77,Ranked personality higher than looks
3,Danish,Men,23,Ranked looks higher than personality
4,Danish,Women,502,Unweighted Sample


In [21]:
circleColor = "#449EA0"

base = (
    alt.Chart(df08w01)
    .transform_filter(filter='datum.Measure=="Ranked personality higher than looks"')
    .encode(
        y=alt.Y("Nationality:N")
        .axis(
            ticks=False,
            labelPadding=10,
            tickBand="extent",
            gridColor="white",
            grid=True,
            gridWidth=5,
            zindex=0,
            domainWidth=2,
            domainColor="black",
        )
        .sort(op="max", field="Value", order="descending")
        .title(""),
    )
)

points = (
    base.transform_calculate(
        Percentage=alt.datum.Value / 100,
    )
    .mark_circle(
        size=150,
        stroke=circleColor,
        strokeWidth=3,
        fillOpacity=1,
    )
    .encode(
        x=alt.X("Value:Q")
        .title("")
        .axis(
            orient="top",
            ticks=False,
            labelExpr='(datum.value==0 || datum.value==50 || datum.value==100) ? datum.label : ""',
            gridColor="white",
            gridWidth=2,
            zindex=0,
            domain=False,
        ),
        fill=alt.Fill("Gender:N")
        .scale(domain=["Men", "Women"], range=[circleColor, "white"])
        .legend(
            orient="top",
            title=None,
        ),
        tooltip=[
            "Nationality:N",
            "Gender:N",
            alt.Tooltip("Percentage:Q", format=".0%"),
        ],
    )
)

lines = (
    base.transform_filter(
        alt.FieldOneOfPredicate(
            field="Nationality", oneOf=["Egyptian", "Saudi Arabian", "Vietnamese"]
        )
    )
    .transform_pivot(
        pivot="Gender",
        groupby=["Nationality"],
        value="Value",
    )
    .mark_rule()
    .encode(x="Men:Q", x2="Women:Q", size=alt.value(2))
)

rule50 = alt.Chart().mark_rule(strokeDash=[4, 4], strokeWidth=1.5).encode(
    x=alt.X(datum=50).axis(grid=True)
)
rule100 = alt.Chart().mark_rule(strokeWidth=2).encode(
    x=alt.X(datum=100)
)

title = alt.Title(
    text=[
        "Across the world women are more likely than men to value",
        "personality over looks",
    ],
    subtitle=[
        "% of people who ranked a romantic partner having a personality they liked as",
        "more important than them being good looking",
    ],
    anchor="start",
    offset=20,
    fontSize=16,
)

rect = alt.Chart().mark_rect(
    fill="white",
    cornerRadius=10,
    strokeWidth=1,
    stroke="black",
)
x1, x2, y1, y2 = 10, 45, "Egyptian", "Saudi Arabian"
rect1 = rect.encode(
    x=alt.datum(x1),
    x2=alt.datum(x2),
    y=alt.datum(y1),
    y2=alt.datum(y2),
)
annotation = alt.Data(values=[{
    'ann1': ['The biggest differences','in opinion between a', 'nation\'s men and',
          'women were in Egypt', 'and Saudi Arabia (both',' 28%)'],
    'ann2': ['Vietnamese men','were the only group','who were more', 'likely to value a','partner\'s looks more','than their','personality']}])
text = alt.Chart(annotation).mark_text(
    align="left",
    baseline="top",
    dx=5, dy=5,
)
text1 = text.encode(
    x=alt.datum(x1),
    y=alt.datum(y1),
    text="ann1:N",
)
arrow1 = alt.layer(
    alt.Chart().mark_rule().encode(
        x=alt.datum(x2),
        x2=alt.datum(65),
        y=alt.datum("French"),
        y2=alt.datum("Egyptian")
    ),
    alt.Chart().mark_point(shape="triangle").encode(
        x=alt.datum(65),
        y=alt.datum("Egyptian"),
    )
)

x_1, x_2, y_1, y_2 = 5, 36, 'UAE', 'Vietnamese'
rect2 = rect.encode(
    x=alt.datum(x_1),
    x2=alt.datum(x_2),
    y=alt.datum(y_1),
    y2=alt.datum(y_2),
)
text2 = text.encode(
    x=alt.datum(x_1),
    y=alt.datum(y_1),
    text="ann2:N",
)

alt.layer(
    rule50,
    rule100,
    rect1,
    text1,
    arrow1,
    rect2,
    text2,
    lines,
    points,
    title=title,
).configure_view(
    strokeOpacity=0,
    fill="#D5D6D8",
).properties(
    width=400,
    height=500,
)

alt.LayerChart(...)